In [2]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

In [115]:
alunos = pd.read_csv("../alunosUFCGAnon.csv")
pre_requisitos = pd.read_csv("../preRequisitos.csv")
periodo_disc = pd.read_csv("../periodoDisciplinas.csv")
# alunos.head(), pre_requisitos.head()

In [114]:
# ajustando data frame que servirá de entrada
alunos_cc = alunos.query("Cod_Curso == 14102100 & Tipo == 'Obrigatória' ")
alunos_cc.sort_values('Media_Disciplina', ascending=False).drop_duplicates(['Nome_Disciplina','Matricula'])
COMPUTACAOtdf = pd.pivot_table(alunos_cc, values = 'Media_Disciplina', index = ['Nome_Disciplina'], columns = 'Matricula')
notas_alunos = alunos_cc
notas_alunos['Matricula'] = alunos_cc['Matricula'].map(lambda x: x.lstrip('B'))
notas_alunos = pd.pivot_table(notas_alunos, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
notas_alunos.head()
# cadeiras que não exigem prerrequisito
sem_prerequisito = ['DIREITO E CIDADANIA','GERÊNCIA DA INFORMAÇÃO','INFORMÁTICA E SOCIEDADE','MATEMÁTICA DISCRETA',
                    'METODOLOGIA CIENTÍFICA','SEMINÁRIOS (EDUCAÇÃO AMBIENTAL)','CALCULO DIFERENCIAL E INTEGRAL I',
                     'PROGRAMAÇÃO I','LABORATÓRIO DE PROGRAMAÇÃO I','ÁLGEBRA VETORIAL E GEOMETRIA ANALÍTICA',
                       'LEITURA E PRODUCAO DE TEXTOS','INTRODUÇÃO A COMPUTAÇÃO']


/home/livia.cavalcanti/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [137]:
# guardandos as regressoes para cada disciplina em um dicionario
regressoes = {}
for disciplina in periodo_disc["DISCIPLINA"]:
    periodo = periodo_disc.loc[periodo_disc["DISCIPLINA"] == disciplina]['PERIODO'].values[0]
    disciplinas_regressao = periodo_disc.loc[periodo_disc["PERIODO"]<=periodo]["DISCIPLINA"].values
    disciplinas_regressao = notas_alunos[disciplinas_regressao].dropna()
    X = disciplinas_regressao.loc[:,disciplinas_regressao.columns != disciplina]
    y = disciplinas_regressao.loc[:,disciplinas_regressao.columns == disciplina]
    regressoes[disciplina] = regressao.fit(X,y)



Nome_Disciplina  CALCULO DIFERENCIAL E INTEGRAL I  \
Matricula                                           
10092                                    6.900000   
10304                                    2.733333   
10504                                    4.650000   
10638                                    7.800000   
10973                                    5.600000   
11044                                    8.700000   
11563                                    8.900000   
11671                                    3.933333   
11776                                    6.100000   
11782                                    8.500000   
11889                                    5.450000   
11954                                    4.033333   
12119                                    7.100000   
12263                                    8.600000   
12477                                    5.800000   
12482                                    7.100000   
12686                                    5.600

Nome_Disciplina  ALGEBRA LINEAR I  CALCULO DIFERENCIAL E INTEGRAL I  \
Matricula                                                             
10092                    6.100000                          6.900000   
10304                    6.200000                          2.733333   
10504                    5.000000                          4.650000   
10638                    8.200000                          7.800000   
10973                    6.700000                          5.600000   
11044                    9.300000                          8.700000   
11563                    7.000000                          8.900000   
11671                    5.900000                          3.933333   
11776                    7.000000                          6.100000   
11782                    7.100000                          8.500000   
11889                    5.500000                          5.450000   
11954                    3.750000                          4.033333   
12119 

Nome_Disciplina  ALGEBRA LINEAR I  CALCULO DIFERENCIAL E INTEGRAL I  \
Matricula                                                             
10092                    6.100000                          6.900000   
10304                    6.200000                          2.733333   
10504                    5.000000                          4.650000   
10638                    8.200000                          7.800000   
10973                    6.700000                          5.600000   
11044                    9.300000                          8.700000   
11563                    7.000000                          8.900000   
11671                    5.900000                          3.933333   
11776                    7.000000                          6.100000   
11782                    7.100000                          8.500000   
11889                    5.500000                          5.450000   
11954                    3.750000                          4.033333   
12119 

Nome_Disciplina  ALGEBRA LINEAR I  CALCULO DIFERENCIAL E INTEGRAL I  \
Matricula                                                             
10092                    6.100000                          6.900000   
10504                    5.000000                          4.650000   
10638                    8.200000                          7.800000   
10973                    6.700000                          5.600000   
11044                    9.300000                          8.700000   
11563                    7.000000                          8.900000   
11671                    5.900000                          3.933333   
11776                    7.000000                          6.100000   
11889                    5.500000                          5.450000   
12263                    9.000000                          8.600000   
12477                    7.300000                          5.800000   
12482                    5.500000                          7.100000   
12686 

Nome_Disciplina  ALGEBRA LINEAR I  ANÁLISE E TÉCNICA DE ALGORITMOS  \
Matricula                                                            
10092                    6.100000                             7.00   
10504                    5.000000                             8.00   
11044                    9.300000                             8.70   
11671                    5.900000                             7.70   
11776                    7.000000                             8.10   
12263                    9.000000                             9.50   
12477                    7.300000                             7.70   
13243                    7.000000                             7.70   
13560                    9.500000                             9.20   
13709                    7.200000                             8.70   
13967                    5.000000                             7.10   
14556                    5.700000                             6.40   
14666               

Nome_Disciplina  ALGEBRA LINEAR I  ANÁLISE E TÉCNICA DE ALGORITMOS  \
Matricula                                                            
10092                    6.100000                             7.00   
10504                    5.000000                             8.00   
10973                    6.700000                             7.60   
11044                    9.300000                             8.70   
11671                    5.900000                             7.70   
11776                    7.000000                             8.10   
12263                    9.000000                             9.50   
12477                    7.300000                             7.70   
13243                    7.000000                             7.70   
13560                    9.500000                             9.20   
13709                    7.200000                             8.70   
13967                    5.000000                             7.10   
14556               

In [23]:
%run util.ipynb


In [143]:
alunos_predicao = notas_alunos.sample(n=5)
for index, aluno in alunos_predicao.iterrows():
    sem_prerequisito = ['DIREITO E CIDADANIA','GERÊNCIA DA INFORMAÇÃO','INFORMÁTICA E SOCIEDADE','MATEMÁTICA DISCRETA',
                        'METODOLOGIA CIENTÍFICA','SEMINÁRIOS (EDUCAÇÃO AMBIENTAL)','CALCULO DIFERENCIAL E INTEGRAL I',
                         'PROGRAMAÇÃO I','LABORATÓRIO DE PROGRAMAÇÃO I','ÁLGEBRA VETORIAL E GEOMETRIA ANALÍTICA',
                           'LEITURA E PRODUCAO DE TEXTOS','INTRODUÇÃO A COMPUTAÇÃO']
    # coletando quais cadeiras o aluno já foi aprovado
    cadeiras_aluno = aluno
    cadeiras_pagas = cadeiras_aluno.dropna().to_frame()
#     print(cadeiras_aluno)
    cadeiras_pagas = cadeiras_pagas[cadeiras_pagas.iloc[:,0] >= 5]

    cadeira_possivel = []

    # vendo quais cadeiras sem prerrequisito já foram pagas pelo aluno
    for cadeira in sem_prerequisito:
        if not cadeira in cadeiras_pagas.index:
            cadeira_possivel.append(cadeira)
    prox_possiveis_cadeiras = prox_cadeiras_nome(cadeiras_pagas, cadeira_possivel, pre_requisitos)#.loc[0:1]
    
#     print("##PRINT CADEIRAS PAGAS##")
#     print(cadeiras_pagas)
#     print("PRINT CADEIRAS POSSIVEIS")
#     print(prox_possiveis_cadeiras)
    cadeiras_aluno = cadeiras_aluno.fillna(0)
    for cadeira in prox_possiveis_cadeiras:
        periodo = periodo_disc.loc[periodo_disc["DISCIPLINA"] == cadeira]['PERIODO'].values[0]
        disciplinas_regressao = periodo_disc.loc[periodo_disc["PERIODO"]<=periodo]["DISCIPLINA"].values
#         regressao.predict(
        print((cadeiras_aluno.loc[disciplinas_regressao]).index)
        
    
    dict_notas = {}
    
    print("=====================")
#     for cadeira in prox_possiveis_cadeiras:
#         dict_notas[cadeira] = math.trunc(get_nota(COMPUTACAOtdf.T[[cadeira]],k_vizinhos, cadeira)*10)

#     # imprimindo notas preditas
#     list_notas = sorted(dict_notas.items(), key = operator.itemgetter(1), reverse = True)
#     print("Notas do historico do aluno:")
#     print(aluno.T.dropna())
#     print("-----------")
#     print("Notas preditas: ")
#     if not list_notas:
#         print("Sem recomendaçao")
#     else:
#         for cadeira in list_notas: 
#             print(cadeira[0], ": ", cadeira[1]/10)
    
#     print("================ \n")


Index(['ALGEBRA LINEAR I', 'ANÁLISE E TÉCNICA DE ALGORITMOS',
       'BANCO DE DADOS I', 'BANCO DE DADOS II',
       'CALCULO DIFERENCIAL E INTEGRAL I', 'CALCULO DIFERENCIAL E INTEGRAL II',
       'COMPILADORES', 'DIREITO E CIDADANIA', 'ENGENHARIA DE SOFTWARE I',
       'ESTRUTURA DE DADOS E ALGORITMOS', 'FUNDAMENTOS DE FÍSICA CLÁSSICA',
       'FUNDAMENTOS DE FÍSICA MODERNA', 'GERÊNCIA DA INFORMAÇÃO',
       'INFORMÁTICA E SOCIEDADE', 'INTELIGENCIA ARTIFICIAL I',
       'INTERCONEXÃO DE REDES DE COMPUTADORES', 'INTRODUÇÃO A COMPUTAÇÃO',
       'LAB.DE ESTRUTURA DE DADOS E ALGORITMOS',
       'LAB.DE INTERCON.DE REDES DE COMPUTADORES',
       'LAB.DE ORG.E ARQUITETURA DE COMPUTADORES',
       'LABORATÓRIO DE ENGENHARIA DE SOFTWARE', 'LABORATÓRIO DE PROGRAMAÇÃO I',
       'LABORATÓRIO DE PROGRAMAÇÃO II', 'LEITURA E PRODUCAO DE TEXTOS',
       'LÓGICA MATEMÁTICA', 'MATEMÁTICA DISCRETA', 'METODOLOGIA CIENTÍFICA',
       'METODOS ESTATISTICOS', 'ORG.E ARQUITETURA DE COMPUTADORES I',
      